### Proceso river

Puede tardar un rato puesto que en este caso se van a entrenar todos los modelos con el dataset completo.

In [1]:
import pandas as pd
import river
import sys
from river import metrics, preprocessing, forest
    
def features_labels_accommodation(features, labels):
        """Perform accomodation on features and labels. Type casting..."""
        
        features["user"] = float(features["user"])
        features["kernel"] = float(features["kernel"])
        features["idle"] = float(features["idle"])

        features["Main"] = int(features["Main"])
        features["aes"] = int(features["aes"])
        features["bulk"] = int(features["bulk"])
        features["crs"] = int(features["crs"])
        features["kmp"] = int(features["kmp"])
        features["knn"] = int(features["knn"])
        features["merge"] = int(features["merge"])
        features["nw"] = int(features["nw"])
        features["queue"] = int(features["queue"])
        features["stencil2d"] = int(features["stencil2d"])
        features["stencil3d"] = int(features["stencil3d"])
        features["strided"] = int(features["strided"])

        # Get each model label
        labels = [float(labels[key]) for key in labels]

        return features, labels


#
# Dataset
#

# Read dataset
dataset_df = pd.read_pickle("dataset.pkl")

# Extract features
features_df = dataset_df.drop(["Top power", "Bottom power", "Time"], axis=1)

# Extract labels
labels_df = dataset_df[["Top power", "Bottom power", "Time"]]
TP = labels_df.iloc[:, 0] #size 98525
BP = labels_df.iloc[:, 1]
Time = labels_df.iloc[:, 2]

#
# Model Initialization
#

# Initialize PS power model
top_power_model = (
                river.preprocessing.StandardScaler() |
                river.tree.HoeffdingAdaptiveTreeRegressor(
                    max_depth=100,
                    grace_period=50,
                    model_selector_decay=0.05,
                    seed=42
                )
            )

# Initialize PL power models
bottom_power_model = (
                river.preprocessing.StandardScaler() |
                river.tree.HoeffdingAdaptiveTreeRegressor(
                    max_depth=100,
                    grace_period=50,
                    model_selector_decay=0.05,
                    seed=42
                )
            )

# Initialize Execution time model
time_model = tmp_model = river.forest.ARFRegressor(seed=42, max_features=None, grace_period=50, n_models = 5, max_depth=100, model_selector_decay=0.05)

# Create model MAPE metrics
top_power_mape = river.utils.Rolling(river.metrics.MAPE(), window_size=1000)
bottom_power_mape = river.utils.Rolling(river.metrics.MAPE(), window_size=1000)
time_mape = river.utils.Rolling(river.metrics.MAPE(), window_size=1000)

# List of metrics and models
models = [top_power_model, bottom_power_model, time_model]
metrics = [top_power_mape, bottom_power_mape, time_mape]
model_names = ["PS Power", "PL Power", "Execution Time"]

#
# Model Train
#

top_model_metric_history = []
bottom_model_metric_history = []
time_model_metric_history = []

# Loop over the observations
for features, labels in river.stream.iter_pandas(features_df, labels_df, shuffle=False, seed=42):

    # Features and labels accommodation
    features, labels = features_labels_accommodation(features, labels)
    for model_type, (model, metric, label) in enumerate(zip(models, metrics, labels)):
        # Make a prediction
        y_pred = model.predict_one(features)
        # Train the model
        model.learn_one(features, label)
        # Update metric
        metric.update(label, y_pred)
        
        

        if model_type == 0:
            top_model_metric_history.append(metric.get())
        elif model_type == 1:
            bottom_model_metric_history.append(metric.get())
        else:
            time_model_metric_history.append(metric.get())

# Print metrics
for model_name, metric in zip(model_names, metrics):
    print(f"{model_name} - MAPE: {round(metric.get(),2)}%")

c:\Users\Usuario\anaconda3\envs\cnntabular\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


PS Power - MAPE: 3.01%
PL Power - MAPE: 2.67%
Execution Time - MAPE: 41.57%


In [2]:
##############################################################################
##### ADAPTIVE CONTINUOUS LEARNING APPROACH OF TWO LAYER NEURAL NETWORKS #####
##############################################################################

###########################################################
##### DEFINICIÓN DE LIBRERÍAS Y LECTURA BASE DE DATOS #####
###########################################################

import numpy as np
import pandas as pd
import os

dataset = pd.read_pickle("dataset.pkl")

# Path to store obtained results
STORE_PATH = "C:/Users/Usuario/Desktop/Universidad/UPM/TFM/Pruebas iniciales"

os.chdir(STORE_PATH)

# Extract features
features = dataset.drop(["Top power", "Bottom power", "Time"], axis=1)

# Extract labels
labels = dataset[["Top power", "Bottom power", "Time"]]
TP = labels.iloc[:, 0] #size 98525
BP = labels.iloc[:, 1]
Time = labels.iloc[:, 2]

In [3]:
###############################
##### DEFINICIÓN DE REDES #####
###############################

# Listas con los parámetros de los tres mejores modelos de dos capas con dropout de cada tipo de modelo
Model_TP_1 = [16, 0, 8]
Model_TP_2 = [32, 0, 4]
Model_TP_3 = [12, 0, 12]

Model_BP_1 = [32, 0.1, 20]
Model_BP_2 = [32, 0.1, 12]
Model_BP_3 = [28, 0.2, 24]

Model_Time_1 = [32, 0, 20]
Model_Time_2 = [32, 0.1, 24]
Model_Time_3 = [28, 0.1, 16]



In [ ]:
######################################
##### ENTRENAMIENTO DE LAS REDES #####
######################################

from incremental_modeling_functions import train_cont_model_2layer, train_infer_times

# Lista para guardar los resultados
print("Training Top Power models")
resultados_error_cont_Model_TP_1 = train_cont_model_2layer(features, TP, Model_TP_1)
resultados_error_cont_Model_TP_2 = train_cont_model_2layer(features, TP, Model_TP_2)
#resultados_error_cont_Model_TP_3 = train_cont_model_2layer(features, TP, Model_TP_3)

with pd.ExcelWriter("Resultados_entrenamiento_continuo_idle_TP.xlsx") as writer: 
    df_resultados_error_cont_Model_TP_1 = pd.DataFrame(resultados_error_cont_Model_TP_1)
    df_resultados_error_cont_Model_TP_1.to_excel(writer, sheet_name="Model 1")
    df_resultados_error_cont_Model_TP_2 = pd.DataFrame(resultados_error_cont_Model_TP_2)
    df_resultados_error_cont_Model_TP_2.to_excel(writer, sheet_name="Model 2")
    #df_resultados_error_cont_Model_TP_3 = pd.DataFrame(resultados_error_cont_Model_TP_3)
    #df_resultados_error_cont_Model_TP_3.to_excel(writer, sheet_name="Model 3")




print("Training Bottom Power models")
resultados_error_cont_Model_BP_1 = train_cont_model_2layer(features, BP, Model_BP_1)
resultados_error_cont_Model_BP_2 = train_cont_model_2layer(features, BP, Model_BP_2)
#resultados_error_cont_Model_BP_3 = train_cont_model_2layer(features, BP, Model_BP_3)

with pd.ExcelWriter("Resultados_entrenamiento_continuo_idle_BP.xlsx") as writer: 
    df_resultados_error_cont_Model_BP_1 = pd.DataFrame(resultados_error_cont_Model_BP_1)
    df_resultados_error_cont_Model_BP_1.to_excel(writer, sheet_name="Model 1")
    df_resultados_error_cont_Model_BP_2 = pd.DataFrame(resultados_error_cont_Model_BP_2)
    df_resultados_error_cont_Model_BP_2.to_excel(writer, sheet_name="Model 2")
    #df_resultados_error_cont_Model_BP_3 = pd.DataFrame(resultados_error_cont_Model_BP_3)
    #df_resultados_error_cont_Model_BP_3.to_excel(writer, sheet_name="Model 3")

print("Training Time models")
resultados_error_cont_Model_Time_1 = train_cont_model_2layer(features, Time, Model_Time_1)
resultados_error_cont_Model_Time_2 = train_cont_model_2layer(features, Time, Model_Time_2)
#resultados_error_cont_Model_Time_3 = train_cont_model_2layer(features, Time, Model_Time_3)

with pd.ExcelWriter("Resultados_entrenamiento_continuo_idle_Time.xlsx") as writer: 
    df_resultados_error_cont_Model_Time_1 = pd.DataFrame(resultados_error_cont_Model_Time_1)
    df_resultados_error_cont_Model_Time_1.to_excel(writer, sheet_name="Model 1")
    df_resultados_error_cont_Model_Time_2 = pd.DataFrame(resultados_error_cont_Model_Time_2)
    df_resultados_error_cont_Model_Time_2.to_excel(writer, sheet_name="Model 2")
    #df_resultados_error_cont_Model_Time_3 = pd.DataFrame(resultados_error_cont_Model_Time_3)
    #df_resultados_error_cont_Model_Time_3.to_excel(writer, sheet_name="Model 3")


prueba_tiempos_TP = train_infer_times(features, TP, Model_TP_1)
prueba_tiempos_BP = train_infer_times(features, BP, Model_BP_1)
prueba_tiempos_Time = train_infer_times(features, Time, Model_Time_1)
with pd.ExcelWriter("Prueba_tiempos_train_infer.xlsx") as writer: 
    df_prueba_tiempos_TP = pd.DataFrame(prueba_tiempos_TP)
    df_prueba_tiempos_TP.to_excel(writer, sheet_name="Model TP")
    df_prueba_tiempos_BP = pd.DataFrame(prueba_tiempos_BP)
    df_prueba_tiempos_BP.to_excel(writer, sheet_name="Model BP")
    df_prueba_tiempos_Time = pd.DataFrame(prueba_tiempos_Time)
    df_prueba_tiempos_Time.to_excel(writer, sheet_name="Model Time")


Training Top Power models

Continuous training of a two layer neural network: First layer = 16 neurones, Dropout = 0, Second layer = 8 neurones


'\nwith pd.ExcelWriter("Resultados_entrenamiento_continuo_idle_TP.xlsx") as writer: \n    df_resultados_error_cont_Model_TP_1 = pd.DataFrame(resultados_error_cont_Model_TP_1)\n    df_resultados_error_cont_Model_TP_1.to_excel(writer, sheet_name="Model 1")\n    #df_resultados_error_cont_Model_TP_2 = pd.DataFrame(resultados_error_cont_Model_TP_2)\n    #df_resultados_error_cont_Model_TP_2.to_excel(writer, sheet_name="Model 2")\n    #df_resultados_error_cont_Model_TP_3 = pd.DataFrame(resultados_error_cont_Model_TP_3)\n    #df_resultados_error_cont_Model_TP_3.to_excel(writer, sheet_name="Model 3")\n\n\n\n\nprint("Training Bottom Power models")\nresultados_error_cont_Model_BP_1 = train_cont_model_2layer(features, BP, Model_BP_1)\n#resultados_error_cont_Model_BP_2 = train_cont_model_2layer(features, BP, Model_BP_2)\n#resultados_error_cont_Model_BP_3 = train_cont_model_2layer(features, BP, Model_BP_3)\n\nwith pd.ExcelWriter("Resultados_entrenamiento_continuo_idle_BP.xlsx") as writer: \n    df_r

In [6]:
 # TODO: Not used on-chip. Handle this properly
from incremental_modeling_functions import rolling_mean, plot_best_models

# Preparamos las medias móviles para plotear
y_values_Model_TP_1 = rolling_mean(resultados_error_cont_Model_TP_1)
#y_values_Model_TP_2 = rolling_mean(resultados_error_cont_Model_TP_2)
#y_values_Model_TP_3 = rolling_mean(resultados_error_cont_Model_TP_3)

y_values_Model_BP_1 = rolling_mean(resultados_error_cont_Model_BP_1)
#y_values_Model_BP_2 = rolling_mean(resultados_error_cont_Model_BP_2)
#y_values_Model_BP_3 = rolling_mean(resultados_error_cont_Model_BP_3)

y_values_Model_Time_1 = rolling_mean(resultados_error_cont_Model_Time_1)
#y_values_Model_Time_2 = rolling_mean(resultados_error_cont_Model_Time_2)
#y_values_Model_Time_3 = rolling_mean(resultados_error_cont_Model_Time_3)

# Ploting up to 4 models, river and three best --> only Model_1 is absolutely necessary to plot
plot_best_models("TP", Model_1 = y_values_Model_TP_1, river_model = top_model_metric_history)
plot_best_models("BP", Model_1 = y_values_Model_BP_1, river_model = bottom_model_metric_history)
plot_best_models("Time", Model_1 = y_values_Model_Time_1, river_model = time_model_metric_history)

KeyboardInterrupt: 